In [0]:
#1
# Install ADAL (Azure Active Directory Authentication Library)
%pip install adal

In [0]:
#Import the tools
#from env import env, not used for this notebook
from src import utils, excel

import openpyxl
import pandas as pd

from pyspark.sql import functions as F
from datetime import datetime
from openpyxl.styles import NamedStyle

In [0]:
# 2 A pop up will require the key code for authentication

# Mart authentication
# Note: 'adal' is deprecated; consider using 'msal' for long-term use.

import adal  # ← Required import

# Set MS context
tenant = "common" 
authority = f"https://login.microsoftonline.com/{tenant}" 
context = adal.AuthenticationContext(authority)

# Client ID for Azure Cloud Shell
client_id = "04b07795-8ddb-461a-bbee-02f9e1bf7b46" 
resource_app_id_url = "https://database.windows.net/" 

# Acquire user code
code = context.acquire_user_code(resource_app_id_url, client_id)
print(code['message'])  # Instructs user to authenticate

# Acquire token with device code
token = context.acquire_token_with_device_code(resource_app_id_url, code, client_id)
access_token = token["accessToken"]



In [0]:
# 3

# Read from mart, note table name is "Schema.TableName"

# JDBC driver and connection info
driver = "com.microsoft.sqlserver.jdbc.spark"
url = "jdbc:sqlserver://udalsqlmartprod.database.windows.net;databaseName=udal-sql-mart-qpat"
table_name = "EROC.PEROC_Initial_Validation"

# Read table into Spark DataFrame
df_raw  = spark.read \
       .format("jdbc") \
       .option("url", url) \
       .option("dbtable", table_name) \
       .option("accessToken", access_token) \
       .option("encrypt", "true") \
       .option("hostNameInCertificate", "*.database.windows.net") \
       .load()

In [0]:
#df_raw.display()

In [0]:
from pyspark.sql import functions as F

# Creating the monthly & Org sheet data frame
df_moved_discharged = (
    df_raw
    .where(F.col("EROC_DerMetric").isin("PIFUSRTMV01", "PIFUSRTDC01"))
    .where(F.col("EROC_DerMonth") > '2023-03-01')
    .where(F.col("EROC_Latest_Flag") == 1)  # confirmed correct
    .groupby(
        "EROC_DerMonth",
        "EROC_DerProviderCode",
        "EROC_DerTFC",
        "RegionName",
        "EROC_DerMetric"
    )
    .agg(F.sum("EROC_Value").alias("Value"))
    .orderBy(
        "EROC_DerMonth",
        "EROC_DerProviderCode"
    )
)

#df_moved_discharged.display()

In [0]:
df_moved_discharged.write.format('delta').mode('overwrite').save('abfss://analytics-projects@udalstdataanalysisprod.dfs.core.windows.net/ElectiveRecovery/Projects/PIFU_Validation')

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")#create the pivoted data frame

df_pivoted = (
    df_moved_discharged
    .groupBy("EROC_DerMonth", "EROC_DerProviderCode")
    .pivot("EROC_DerMetric")  # creates columns for each metric
    .agg(F.first("Value"))    # or F.sum if there are multiple rows
    .na.fill(0)  # replace nulls with 0
    .orderBy("EROC_DerMonth", "EROC_DerProviderCode")
)

df_pivoted.display()

In [0]:
#Creating the chart 
from pyspark.sql.functions import to_date, col
from datetime import datetime
import plotly.express as px

# Step 0: Optional cleanup (reset widgets for proper order — safe in dashboards)
dbutils.widgets.removeAll()

# Step 1: Convert date string to proper date type
df_moved_discharged = df_moved_discharged.withColumn(
    "EROC_DerMonth", to_date(col("EROC_DerMonth"), "yyyy-MM-dd")
)

# Step 2: Register the DataFrame as a temp SQL view
df_moved_discharged.createOrReplaceTempView("moved_discharged")

# Step 3: Get base DataFrame with valid rows
df_line_chart = spark.sql("""
    SELECT
        EROC_DerMonth,
        EROC_DerProviderCode,
        EROC_DerMetric,
        Value
    FROM
        moved_discharged
    WHERE
        EROC_DerMonth IS NOT NULL
        AND EROC_DerProviderCode IS NOT NULL
        AND EROC_DerMetric IS NOT NULL
""")

# Step 4: Fetch distinct values and date range
valid_providers = sorted(set(str(p[0]) for p in df_line_chart.select("EROC_DerProviderCode").distinct().collect()))
valid_metrics = sorted(set(str(m[0]) for m in df_line_chart.select("EROC_DerMetric").distinct().collect()))
min_date, max_date = df_line_chart.selectExpr("min(EROC_DerMonth)", "max(EROC_DerMonth)").first()

In [0]:
dbutils.widgets.removeAll()

In [0]:
#Step 5 # 1. Provider dropdown
valid_providers = sorted(set(str(p[0]) for p in df_line_chart.select("EROC_DerProviderCode").distinct().collect()))
dbutils.widgets.dropdown("1_Provider", "All", ["All"] + valid_providers)

# 2. Metric dropdown
valid_metrics = sorted(set(str(m[0]) for m in df_line_chart.select("EROC_DerMetric").distinct().collect()))
dbutils.widgets.dropdown("2_Metric", "All", ["All"] + valid_metrics)

# 3. Start date text box
min_date = df_line_chart.selectExpr("min(EROC_DerMonth)").first()[0]
dbutils.widgets.text("3_StartDate", min_date.strftime("%Y-%m-%d"))

# 4. End date text box
max_date = df_line_chart.selectExpr("max(EROC_DerMonth)").first()[0]
dbutils.widgets.text("4_EndDate", max_date.strftime("%Y-%m-%d"))

In [0]:


# Step 6: Get widget values
selected_provider = dbutils.widgets.get("1_Provider")
selected_metric = dbutils.widgets.get("2_Metric")
start_date_str = dbutils.widgets.get("3_StartDate")
end_date_str = dbutils.widgets.get("4_EndDate")

# Step 7: Validate and parse dates
def safe_parse(date_str, fallback):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return fallback

start_date = safe_parse(start_date_str, min_date)
end_date = safe_parse(end_date_str, max_date)

# Step 8: Apply filtering
df_filtered = df_line_chart.filter((col("EROC_DerMonth") >= start_date) & (col("EROC_DerMonth") <= end_date))

if selected_provider != "All" and selected_provider in valid_providers:
    df_filtered = df_filtered.filter(col("EROC_DerProviderCode") == selected_provider)
if selected_metric != "All" and selected_metric in valid_metrics:
    df_filtered = df_filtered.filter(col("EROC_DerMetric") == selected_metric)

# Step 9: Pivot and display
df_pivot = df_filtered.groupBy("EROC_DerMonth").pivot("EROC_DerMetric").sum("Value").orderBy("EROC_DerMonth")
display(df_pivot)

# Step 10: Optional Plotly line chart for selected metric
if selected_metric != "All":
    pdf = df_filtered.select("EROC_DerMonth", "Value").orderBy("EROC_DerMonth").toPandas()
    fig = px.line(
        pdf,
        x="EROC_DerMonth",
        y="Value",
        title=f"{selected_metric} Trend for {selected_provider}" if selected_provider != "All" else f"{selected_metric} Trend (All Providers)"
    )
    fig.show()

Databricks visualization. Run in Databricks to view.